# Description

Exploring a subset of the Enron emails. Done in a single day as part of a coding challenge/interview challenge.

If I had more time, I'd like to 
 * improve how I imported/tokenized the data, and reorganize the notebook so tokenizing happens upfront. I left it as-is to demonstrate my thought process and iterations as I worked.
 * explore the dataset outside of the NLP/Information Storage and Retrieval space / being a little less tunnel-visioned on that particular aspect

# Imports and boilerplate

In [1]:
import pandas as pd
import spacy
from spacy.lang.en.examples import sentences
nlp = spacy.load('en_core_web_sm')
from collections import Counter
from itertools import chain

# Data import

In [2]:
df_orig = pd.read_csv('enron_test.csv', parse_dates=['Date', 'new_date'])
# Date formats are recorded as 5/14/2001  23:39:00, which is MM/DD/YYYY. This should be the default arrangement
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1000 non-null   object        
 1   From      1000 non-null   object        
 2   To        995 non-null    object        
 3   Subject   698 non-null    object        
 4   content   1000 non-null   object        
 5   new_date  1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 47.0+ KB


In [3]:
df = df_orig.copy(deep=True)

In [4]:
df['new_date'][0].month #more sancheck

5

## Converting string "frozenset" into actual frozenset

In [5]:
df['From'][0] #== "frozenset({'phillip.allen@enron.com'})"
# type(df['From'][0]) == str

"frozenset({'phillip.allen@enron.com'})"

In [6]:
df['From'] = df_orig['From'].apply(lambda x: eval(x), 'columns') #column axis, aka apply to each row
# yes eval in production code is hideously dangerous, good thing this isn't production code
type(df['From'][0]) #frozenset, as desired

frozenset

In [7]:
type(df['To'][188]) #some To entries are null; for example this one is nan

float

In [8]:
df['To'] = df_orig['To'].apply(lambda x: eval(x) if type(x) == type("a") else {}, 'columns') 
# dunno how to feel about this lambda. might rewrite to use pd.isnull
df['To'][227]

frozenset({'frank.ermis@enron.com',
           'jane.tholt@enron.com',
           'jay.reitmeyer@enron.com',
           'keith.holst@enron.com',
           'matthew.lenhart@enron.com',
           'mike.grigsby@enron.com',
           'monique.sanchez@enron.com',
           'randall.gay@enron.com',
           'steven.south@enron.com',
           'susan.scott@enron.com',
           'tori.kuykendall@enron.com'})

## Working with forwarded emails

Some emails were forwarded; for example, this one:

```
"---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 
01:42 PM ---------------------------


""Buckner, Buck"" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM
To: ""'Pallen@Enron.com'"" <Pallen@Enron.com>
cc:  
Subject: FW: fixed forward or other Collar floor gas price terms


Phillip,
"
```

It may be worth creating some additional columns, if I have time later:
* One to indicate if the email was forwarded (boolean true/false)
* One to strip the "forwarded" boilerplate text (if necessary), leaving the actual message behind.
  * Why strip the "forwarded" text? Because with the full text, the entity "Phillip K Allen/HOU/ECT" shows up a whole lot.
  * Why keep the other text? One could argue that a forwarded message, being meant for a new audience, carries as much weight as the original message (even if the fwd'd message is literally the same as the original)

In [9]:
df_orig['Subject'][9].lower()[:3]

'fw:'

In [10]:
df['Forwarded'] = df_orig['Subject'].apply(lambda x: True if (pd.notna(x) and x.lower()[:3] == 'fw:') else False)
# Creates the "Forwarded" boolean column

In [11]:
df['Forwarded'][9]

True

## Some additional data cleaning

In [12]:
df['content2'] = df_orig['content'].apply(lambda x: x.lower().replace("\n", " ") if pd.notna(x) else None)
#Lowercases everything. Replaces newlines with spaces.

In [13]:
df['content2']

0                                here is our forecast   
1      traveling to have a business meeting takes the...
2                         test successful.  way to go!!!
3      randy,   can you send me a schedule of the sal...
4                    let's shoot for tuesday at 11:45.  
                             ...                        
995    jacques,  still trying to close the loop on th...
996    larrry,  i realize you are disappointed about ...
997    ---------------------- forwarded by phillip k ...
998    jacques,  i think we reached an agreement with...
999    ---------------------- forwarded by phillip k ...
Name: content2, Length: 1000, dtype: object

# Term Frequency

Exploring how often certain terms or topics come up in the email bodies. Goal was to find the most discussed (and in some sense, most important) terms/topics.

In [14]:
doc_0 = nlp(sentences[0])
doc_2 = nlp(sentences[2])

combined_entities = doc_0.ents + doc_2.ents 
texts = [x.text for x in combined_entities]
Counter(texts)
# Mostly I just wanted to make sure that this would work

Counter({'Apple': 1, 'U.K.': 1, '$1 billion': 1, 'San Francisco': 1})

## Named entity recognition on the unedited emails


In [15]:
df['Entities'] = df['content'].apply(lambda x: nlp(x).ents)
# warning - takes a while

In [16]:
df['Entities']

0                                                     ()
1                                            ((Austin),)
2                                                     ()
3                                   ((Randy), (Phillip))
4                                   ((Tuesday), (11:45))
                             ...                        
995    ((Jacques), (15,000), (today), (tomorrow), (Ph...
996                                ((15,000), (Phillip))
997    ((Phillip, K, Allen, /, HOU, /, ECT), (12/06/2...
998    ((Jacques), (George), (Larry), (One), (15,000)...
999    ((Phillip, K, Allen, /, HOU, /, ECT), (03/15/2...
Name: Entities, Length: 1000, dtype: object

In [17]:
combined_entities = list(chain.from_iterable(df['Entities'])) 
# couldn't find a "reduce" function in pandas (as map-reduce), used this instead https://stackoverflow.com/a/35005105
print(combined_entities[0].text, combined_entities[0].label_)

Austin PERSON


In [18]:
combined_entities_pairs = [(x.text, x.label_) for x in combined_entities]
# Convert things into pair form so they can be sorted by Counter() -- 
# The later approach doesn't need Counter, admittedly
combined_entities_pairs[0]

('Austin', 'PERSON')

In [19]:
#texts = [x.text for x in combined_entities]
#Counter(texts).most_common(10)

In [20]:
Counter(combined_entities_pairs).most_common(20)

[(('09', 'CARDINAL'), 431),
 (('Phillip', 'PERSON'), 382),
 (('Phillip K Allen/HOU/ECT', 'PERSON'), 351),
 (('HOU/ECT@ECT', 'ORG'), 270),
 (('NA', 'ORG'), 204),
 (('20', 'CARDINAL'), 125),
 (('Phillip Allen', 'PERSON'), 116),
 (('Phillip K Allen/HOU/ECT@ECT\n', 'PERSON'), 116),
 (('today', 'DATE'), 105),
 (('Enron', 'ORG'), 79),
 (('Phillip K Allen/HOU/ECT@ECT', 'PERSON'), 75),
 (('3', 'CARDINAL'), 71),
 (('1', 'CARDINAL'), 70),
 (('Lucy', 'PERSON'), 70),
 (('tomorrow', 'DATE'), 65),
 (('2', 'CARDINAL'), 63),
 (('Jeff', 'PERSON'), 62),
 (('one', 'CARDINAL'), 59),
 (('two', 'CARDINAL'), 58),
 (('ISO', 'ORG'), 54)]

**What does this tell us?**

It mostly tells us that Phillip K Allen was involved in a lot of these communications...which is not very surprising, considering so many of the emails were sent from, or directed to, Phillip. (In fact that's probably how this subset was sliced.)

There's a lot of cardinals that can be stripped here, and specific token types (e.g. `ORG` for Organization) might be worth exploring further.

In [21]:
#combined_entities_pairs_no_cardinals = filter(lambda x: x[1] != 'CARDINAL', combined_entities_pairs)
#Counter(combined_entities_pairs_no_cardinals).most_common(20)

In [22]:
combined_entities_pairs_orgs = filter(lambda x: x[1] == 'ORG', combined_entities_pairs)
Counter(combined_entities_pairs_orgs).most_common(20)

[(('HOU/ECT@ECT', 'ORG'), 270),
 (('NA', 'ORG'), 204),
 (('Enron', 'ORG'), 79),
 (('ISO', 'ORG'), 54),
 (('HOU/EES@EES', 'ORG'), 49),
 (('FERC', 'ORG'), 43),
 (('HOU/ECT', 'ORG'), 35),
 (('OWA', 'ORG'), 35),
 (('LLC', 'ORG'), 34),
 (('Enron North America Corp.', 'ORG'), 31),
 (('MW', 'ORG'), 31),
 (('HOU/ECT\n', 'ORG'), 30),
 (('Yahoo', 'ORG'), 30),
 (('AES', 'ORG'), 28),
 (('HOU/ECT@ECT\n', 'ORG'), 26),
 (('TDS', 'ORG'), 24),
 (('TX', 'ORG'), 21),
 (('p&l', 'ORG'), 21),
 (('SoCal', 'ORG'), 21),
 (('PDX/ECT@ECT', 'ORG'), 20)]

**What does this tell us?**

Some of the organizations named and involved in these emails. Some (e.g. `HOU/ECT@ETC`) are internal. Others (e.g. `yahoo`, `socal`) are not. Some (`AES`) are stock ticker symbols for other companies (which makes sense given Enron's financial dealings). 

This also calls to attention
* the fact that I need to strip newlines from some of these, as `HOU/ECT@ECT` and `HOU/ECT@ETC\n` were considered separate entities. And do a lot more data cleanup (e.g. forced lowercasing) 
  * done
* the fact that AES is mentioned 28 times ... but only in 2 emails (one of which was a forward); therefore, putting AES as one of the "most frequent" topics mentioned isn't really representative of the underlying data.
  * This particular insight (term frequency over the bag-of-words, vs *document frequency* (a 1/0 if the document contains/doesn't contain the term) motivated me to study IDF scores. 
  * However, IDF is *inverse* document frequency, prioritizing rarewords because they're interpreted as "more informative" (like "arachnophobia" being present in a webpage says more than "the" being present). 
  * Normal document frequency might be desired (and was calculated in a middle step)... in some sense, this was unnecessary for me to do by hand, as I could have leveraged spaCy to handle it automatically (by counting all things, not just Named Entities) -- however, it was nice to have in a vectorized/numpy-compatible form.

## Named entity recognition on (slightly) cleaned up data

In [23]:
df['entities2'] = df['content2'].apply(lambda x: nlp(x).ents)
# Again, will take a while

In [24]:
combined_entities_2 = list(chain.from_iterable(df['entities2'])) 
combined_entities_pairs_2 = [(x.text, x.label_) for x in combined_entities_2]
Counter(filter(lambda x: x[1] != 'CARDINAL', combined_entities_pairs_2)).most_common(20)

[(('phillip k allen/hou/', 'PERSON'), 284),
 (('phillip k allen/hou/ect@ect', 'PERSON'), 195),
 (('today', 'DATE'), 125),
 (('enron', 'ORG'), 108),
 (('california', 'GPE'), 84),
 (('phillip k allen/hou/ect', 'PERSON'), 79),
 (('daily', 'DATE'), 76),
 (('hou/ect@ect', 'ORG'), 67),
 (('first', 'ORDINAL'), 63),
 (('tomorrow', 'DATE'), 61),
 (('friday', 'DATE'), 57),
 (('larry', 'PERSON'), 56),
 (('hr/corp/enron@enron', 'ORG'), 51),
 (('wednesday', 'DATE'), 50),
 (('monday', 'DATE'), 48),
 (('monthly', 'DATE'), 47),
 (('ferc', 'ORG'), 47),
 (('jeff', 'PERSON'), 47),
 (('thursday', 'DATE'), 44),
 (('tim belden/hou/ect@ect', 'PERSON'), 40)]

# TF/IDF 

Or some other algorithm that deals with *non*-named entities and concepts, e.g. "trades" and "company" and the like.

## A TF/IDF matrix

More useful for information storage/retrieval contexts than for strictly analytical contexts. However, I thought it was neat.

In [25]:
import numpy as np

In [34]:
def valid_term(x):
    is_stop = x.is_stop
    is_blank = (len(x.lemma_.strip()) == 0)
    is_single_char = (len(x.lemma_) == 1)
    is_punct = (x.is_punct or x.text in ['<', '>'])
    is_fwd = (x.text[:5] == "-----") #dunno if actually needed or if lemma takes care of it. w/e tho
    #is_number = x.lemma_.isnumeric() #might want to keep numbers in emails about finance data...
    return not (is_stop or is_blank or is_single_char or is_punct or is_fwd)

In [35]:
import string

punct_reference = string.punctuation.replace("'", "").replace("@", "") #want apostrophes for slang, @ for email addresses
punct_reference

'!"#$%&()*+,-./:;<=>?[\\]^_`{|}~'

In [36]:
def strip_punct(x):
    translation_table = str.maketrans('', '', punct_reference)
    return x.translate(translation_table)

strip_punct(")heya")
# from https://machinelearningmastery.com/clean-text-machine-learning-python/

'heya'

In [37]:
def spacy_tokenizer(document):
    tokens = nlp(document)
    tokens = [strip_punct(token.lemma_) for token in tokens if valid_term(token)]
    return tokens

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
# thanks to https://towardsdatascience.com/lovecraft-with-natural-language-processing-part-3-tf-idf-vectors-8c2d4df98621 for the tutorial

In [39]:
tfidf_vectorizer = TfidfVectorizer(input = 'content', tokenizer = spacy_tokenizer)

In [40]:
result = tfidf_vectorizer.fit_transform(df['content2'])
# Might be waiting here for a while

In [41]:
tfidf_vectorizer.get_feature_names()[:10]

c:\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['',
 "'em",
 '0',
 '00',
 '000',
 '000000000000935',
 '000000000001282',
 '000000000009659',
 '000000000021442',
 '000119']

In [42]:
df_results = pd.DataFrame(
    result.todense().tolist(), columns=tfidf_vectorizer.get_feature_names()
)
    

In [43]:
df_results

,,'em,0,00,000,000000000000935,000000000001282,000000000009659,000000000021442,000119,...,ziegelaar,zimin,zisman,zivic,zone,zoning,zoom,zub,zufferli,zuniga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calculate IDF

Broadly speaking, a measure of how "informative" a term is. More useful for the given prompt "What are the most commonly mentioned topics in the email body?"

* Calculate *document frequency* $df_t$
* Use numpy to handle the conversion from $df_t$ to $idf_t$

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
count_vectorizer = CountVectorizer(input = 'content', tokenizer = spacy_tokenizer)

In [46]:
result_2 = count_vectorizer.fit_transform(df['content2'])
# takes a hot minute

In [47]:
count_vectorizer.get_feature_names_out()

array(['', "'em", '0', ..., 'zub', 'zufferli', 'zuniga'], dtype=object)

In [48]:
result_2

<1000x8770 sparse matrix of type '<class 'numpy.int64'>'
	with 53440 stored elements in Compressed Sparse Row format>

In [49]:
result_2[:,1] #1000 rows (documents), 1 word. testing some stuff out...

<1000x1 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [50]:
result_2[:,6700].count_nonzero()

#loginjhtml has an actual frequency of 2 and a calculated frequency of 2 as well.
#well, it's actually 0, but it's "login.jhtml" in the corpus.

12

In [51]:
count_vectorizer.get_feature_names_out()[6700]

'potentially'

In [52]:
counts = np.apply_along_axis(np.count_nonzero, 0, result_2.todense())
len(list(counts)) #check it's the right axis

8770

In [53]:
counts_df = pd.DataFrame(
    counts,
    index = count_vectorizer.get_feature_names_out(),
    columns = ['Document Frequency']
)
counts_df

,Document Frequency
,7
'em,2
0,5
00,9
000,3
...,...
zoning,6
zoom,1
zub,2
zufferli,4


In [54]:
counts_df.sort_values(by = ['Document Frequency'], ascending=False)

,Document Frequency
phillip,732
allen,406
forward,360
hou,342
subject,340
...,...
katie,1
prevail,1
advertiser,1
advertise,1


## Convert document frequency matrix to IDF scores

In [55]:
temp_shape = counts.shape
temp_shape

(8770,)

In [56]:
idf_scores = np.log10(np.divide(np.full(temp_shape,1000), counts))

In [57]:
idf_scores_df = pd.DataFrame(
    idf_scores,
    index = count_vectorizer.get_feature_names_out(),
    columns = ['IDF Score']
)
idf_scores_df

,IDF Score
,2.154902
'em,2.698970
0,2.301030
00,2.045757
000,2.522879
...,...
zoning,2.221849
zoom,3.000000
zub,2.698970
zufferli,2.397940


In [58]:
idf_scores_df.idxmax()
# gets the index of the cell with the highest IDF score.

IDF Score    000000000009659
dtype: object

In [59]:
idf_scores_df.sort_values(by = ['IDF Score'], ascending=False)

,IDF Score
portion09of,3.000000
film,3.000000
aimster,3.000000
ain,3.000000
ain@worldnetattnet,3.000000
...,...
subject,0.468521
hou,0.465974
forward,0.443697
allen,0.391474


# Use Cases/further explorations

The Enron dataset has had a lot of historical usages, some of which include
* as a training dataset for spam email classifiers
* as a training dataset for predictive text algorithms
* identifying financial fraud during the actual Enron case, aggregating actual trades and transactions made
* classifying some emails as relevant to financial fraud, and others as irrelevant
* visualizing connections between the various employees of Enron, and which ones formed cliques/social circles